{/* cspell:ignore webkitallowfullscreen allowfullscreen frameborder referrerpolicy Lipinska Pellow Jarman Tilly Ardle Expressibility Arrasmith mcscf chmax Dmax vmax */}

# Classical Optimizers

## What is an optimizer?

Victoria Lipinska tells us about classical optimizers, and how they function as part of VQE.

You will hear about a few example optimizers and how they perform in the presence and absence of noise.

<iframe width="690" height="390" src="https://video.ibm.com/embed/recorded/132414916"></iframe>

**References**

- [A Comparison of Various Classical Optimizers for a Variational Quantum Linear Solver, Pellow-Jarman, et al.](https://arxiv.org/pdf/2106.08682.pdf)
- [The Variational Quantum Eigensolver: A review of methods and best practices, Tilly, et al.](https://arxiv.org/abs/2111.05176)
- [Quantum computational chemistry, McArdle, et al.](https://arxiv.org/pdf/1808.10402.pdf)
- [Barren plateaus in quantum neural network training landscapes, McClean, et al.](https://arxiv.org/pdf/1803.11173.pdf)
- [Connecting Ansatz Expressibility to Gradient Magnitudes and Barren Plateaus, Holmes, et al.](https://journals.aps.org/prxquantum/pdf/10.1103/PRXQuantum.3.010313)
- [Effect of barren plateaus on gradient-free optimization, Arrasmith, et al.](https://arxiv.org/pdf/2011.12245.pdf)

In [ ]:
from qiskit.quantum_info import SparsePauliOp
import matplotlib.pyplot as plt
import numpy as np


def cholesky(V, eps):
    # see https://arxiv.org/pdf/1711.02242.pdf section B2
    # see https://arxiv.org/abs/1808.02625
    # see https://arxiv.org/abs/2104.08957
    no = V.shape[0]
    chmax, ng = 20 * no, 0
    W = V.reshape(no**2, no**2)
    L = np.zeros((no**2, chmax))
    Dmax = np.diagonal(W).copy()
    nu_max = np.argmax(Dmax)
    vmax = Dmax[nu_max]
    while vmax > eps:
        L[:, ng] = W[:, nu_max]
        if ng > 0:
            L[:, ng] -= np.dot(L[:, 0:ng], (L.T)[0:ng, nu_max])
        L[:, ng] /= np.sqrt(vmax)
        Dmax[: no**2] -= L[: no**2, ng] ** 2
        ng += 1
        nu_max = np.argmax(Dmax)
        vmax = Dmax[nu_max]
    L = L[:, :ng].reshape((no, no, ng))
    print(
        "accuracy of Cholesky decomposition ",
        np.abs(np.einsum("prg,qsg->prqs", L, L) - V).max(),
    )
    return L, ng


def identity(n):
    return SparsePauliOp.from_list([("I" * n, 1)])


def creators_destructors(n, mapping="jordan_wigner"):
    c_list = []
    if mapping == "jordan_wigner":
        for p in range(n):
            if p == 0:
                ell, r = "I" * (n - 1), ""
            elif p == n - 1:
                ell, r = "", "Z" * (n - 1)
            else:
                ell, r = "I" * (n - p - 1), "Z" * p
            cp = SparsePauliOp.from_list([(ell + "X" + r, 0.5), (ell + "Y" + r, -0.5j)])
            c_list.append(cp)
    else:
        raise ValueError("Unsupported mapping.")
    d_list = [cp.adjoint() for cp in c_list]
    return c_list, d_list

Now to define our Hamiltonian, we will use PySCF exactly as in the previous example, but now we will include a variable "x" to play the role of our interatomic distance. This will return the core energy, single-electron energy, and two-electron energies as before.